# 목표

    - 국내 미디어 분야 점유율이 가장 높은 youtube에서 필요한 정보 수집
    - 특정 키워드 검색
        - 오늘, 업로드 순으로 정렬
        - https://www.youtube.com/results?search_query=검색어&sp=CAISBAgCEAE%253D
        - 검색어는 URL 인코딩 처리 필요
        - 썸네일, 영상주소, 기타 메타 데이터(제목, ....)    <----- 지금은 여기까지만 밑에는 프로젝트에서
            - 영상 방문
                - 댓글, 해시태그, 좋아요수, 조회수,
            - 썸네일
                - 글자인식 -> 정보추출(텍스트 인식):딥러닝 -> 정보추출

# 초기접속

In [1]:
# 접속까지 준비
import pandas as pd
import sys
from selenium import webdriver as wd
import random
import time
import os
import sys
import urllib

driver = wd.Chrome('./chromedriver.exe')

keyword = urllib.parse.quote("손흥민")

target_site = f'https://www.youtube.com/results?search_query={keyword}&sp=CAISBAgCEAE%253D'
driver.get(target_site)


# 데이터 더 불러오기
    
    - 스크롤을 진행하여 ajax가 작동(더보기) 모든 데이터가 올라오기를 기다린다.
    - 스크롤은 JS를 강제로 발동시켜서 화면 이동을 시도한다.
        - 셀레니움에서 js가동시키기
    - 회수를 알 수 없으니, 대략 100~1000회 진행한다.

In [2]:

for n in range(30):
    driver.execute_script('''
        // 자바스크립트 작성
        window.scrollBy(0,2000)  
    ''')
    time.sleep(2)

In [3]:
# 게시물별 단위 조사
videos = driver.find_elements_by_tag_name('ytd-video-renderer')
len(videos)

149

In [32]:
import os, os.path
import urllib.request as req

def thumbnail_download( url_src:str, save_path='./data/youtube/thumbnail' ) -> None:    
    if not url_src:return # 이미지 경로값이 없으면 중단
    if not os.path.exists(save_path):
        print(f'요청하신 경로 {save_path} 은 없는 디렉토리 입니다.')
        os.makedirs(save_path)
    
    filename = url_src.split('?')[0].split('/')
    filename = f'{filename[-2]}_{filename[-1]}'    
    print( filename )
    req.urlretrieve( url_src, save_path + "/" + filename )

In [33]:
# 진행율 
from tqdm import tqdm_notebook

# enumerate() => 인덱스, 맴버 : 순서대로 추출
# tqdm_notebook() => 진행율
for idx, video in enumerate( tqdm_notebook(videos) ):
    
    # video 객체 하위에서만 탐색
    #print( video.find_element_by_id('video-title').get_attribute('title') )2
    # 썸네일(링크주소->원하는 위치에 다운로드) 이미지 주소
    img_src = video.find_element_by_css_selector('#thumbnail img').get_attribute('src')
    print( idx, img_src )
    # 썸네일 다운로드
    thumbnail_download( img_src )
    # 영상 링크
    # 기타 하고 싶은 만큼 추출 진행
    # 요소 밑에 텍스트 뽑을때
    # xxx.text 않되면 xxx.get_attribute('innerHTML')
    # 5개만 샘플로 확인
    #if idx>=5:
    #    break

C:\Users\USER\AppData\Local\Temp\ipykernel_16908\2987280630.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, video in enumerate( tqdm_notebook(videos) ):


  0%|          | 0/149 [00:00<?, ?it/s]

0 https://i.ytimg.com/vi/QJyooFDVPDo/hq2.jpg?sqp=-oaymwEdCMoBEOgCSFXyq4qpAw8IARUAAIhCcAHAAQbQAQE=&rs=AOn4CLDU9TupqPIgET_mLRt6UJ78wm7NCw
QJyooFDVPDo_hq2.jpg
1 https://i.ytimg.com/vi/40BY02K_tNw/hq720_2.jpg?sqp=-oaymwEdCJUDENAFSFXyq4qpAw8IARUAAIhCcAHAAQbQAQE=&rs=AOn4CLC5GY6ltQ-tvrAKQSLtNapOr7Cc4Q
40BY02K_tNw_hq720_2.jpg
2 https://i.ytimg.com/vi/D9mxCZr-Rkw/hq720.jpg?sqp=-oaymwEcCNAFEJQDSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCMclWmqewojHcNQcIVmp56wqBJHA
D9mxCZr-Rkw_hq720.jpg
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 None
11 None
12 None
13 None
14 None
15 None
16 None
17 https://i.ytimg.com/vi/xfrLOONvCNk/hq720_2.jpg?sqp=-oaymwEdCJUDENAFSFXyq4qpAw8IARUAAIhCcAHAAQbQAQE=&rs=AOn4CLAOmrJo4GzjrdHFIwHBrtiJ-_SPiw
xfrLOONvCNk_hq720_2.jpg
18 https://i.ytimg.com/vi/kUZ-jbfNRD8/hq2.jpg?sqp=-oaymwEdCMoBEOgCSFXyq4qpAw8IARUAAIhCcAHAAQbQAQE=&rs=AOn4CLDKiSY7dp2DxUjWScpi2jUdHiEkRA
kUZ-jbfNRD8_hq2.jpg
19 https://i.ytimg.com/vi/DKPBOpkZJ6w/hq720.jpg?sqp=-oaymwEcCNAFEJQDSFXyq4qpAw4IARUAAIhCGAFwA

In [27]:
# 한개 예시
# 인터넷상에 존재하는 바이너리 파일들을 내 PC에 특정 위치에 저장하는 방식
img_url = 'https://i.ytimg.com/vi/QJyooFDVPDo/hq2.jpg?sqp=-oaymwEdCMoBEOgCSFXyq4qpAw8IARUAAIhCcAHAAQbQAQE=&rs=AOn4CLDU9TupqPIgET_mLRt6UJ78wm7NCw'
# 필요 모듈
import os, os.path
#진행율 보기
from tqdm import tqdm_notebook
# 요청
import urllib.request as req
# 저장위치 생성(없으면 디렉토리 생성) 혹은 지정
# 현재상태에서 파일을 생성하면 현재 작성중인 파일을 기반으로 경로를 계산해서 생성된다.
save_path = './data/youtube/thumbnail'
# 해당 디렉토리가 없는가?
if not os.path.exists(save_path):
    print(f'요청하신{save_path} 은 없는 디렉토리 입니다.')
    os.makedirs(save_path)

# 저장
# 원격지에 있는 리소스를 로컬 PC상에 지정한 위치에 읽어서 바로 저장
# 데이터 수집 레벨에 관계없이 사용가능
# 개선사항 -> 파일명을 원본 주소에서 획득 혹은 해시값(중복되지 않는값)으로 관리
# 파일 확장자는 반드시 url에서 획득 => 이미지 타입 변경 => 이미지 라이브러리 사용(논외)

# print(img_url.split('?')[0][len('hq2.jpg')*-1:] )
# print(img_url.split('?')[0].split('/')[-1])  -> 어떤 url이 와도 파일명 찾기
filename =img_url.split('?')[0].split('/')[-1]
req.urlretrieve(img_url,save_path + "/" + "filename" )  #이게 




('./data/youtube/thumbnail/filename',
 <http.client.HTTPMessage at 0x1812d22f7c0>)

In [24]:
print(img_url.split('?')[0].split('/')[-1])

hq2.jpg
